## Robô com os modelos de predição utilizados para os alunos abaixo:
 **📚 MBA - Business Analytics & Big Data - T9 - FGV Paulista**
 
 ↪Bruno Pioli
 
 ↪Guilherme Alves
 
 ↪João Pedro Dannemann
 
 ↪Ricardo Santos

# MODELO 2> ANÁLISE LSTM VETORIAL: 2 PRÓXIMOS VALORES PREVISTOS

Como explicado no arquivo robo_BGJP.py, este modelo prevê os dois próximos valores de cada moeda e:
    
- Se a expectativa for de subida (maior que +0,05%)  **OU** ambas as previsões aumentaram de valor:
    Se possuirmos dinheiro em caixa, o modelo compra o equivalente proporcional à subida, limitado entre USD 10 e USD 1000
- Se a expectativa for de queda (menor que -0,05%) **OU** ambas as previsões reduziram de valor:
    Se possuirmos moeda em caixa, o modelo vende o equivalente proporcional à descida, limitado entre USD 10 e USD 1000


In [ ]:
#Bibliotecas Necessárias:
from numpy import array
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
import time
from robo_BGJR import api_get, api_post, get_result, compute_quantity, how_much_i_have, LSTMvetor, split_sequenceVet

### Modelo 2: LSTM com saída Vetorial

In [ ]:
# Carrega o modelo criado no notebook anterior
token = '039ddfe6fc4ef0f'
ticker=['BTCUSDT','DOGEUSDT','ETHUSDT']
valor_compra_venda = 500
x=1 # a variável x será usada para contar a quantidade de rodadas feitas

In [ ]:
while True:
# Loop Interno 1: Obter a tabela das ultimas 500 cotações de cada moeda e:
# Cria 2 Arrays: 1. Ultimo Valor de cada moeda 2: Próximo valor estimado
    proxvalor=[]
    ultimovalor=[]
    tendencia=[]
    qtdade=[]
    previsoes=pd.DataFrame()
    qteprev = 2 
    for i in range(3):
        df = api_post('cripto_quotation', {'token': token, 'ticker': ticker[i]})
        prov=LSTMvetor(df['close'],3,qteprev)
        proxvalor=pd.DataFrame(prov)
        previsoes=previsoes.append(proxvalor)
        proxvalor=[]
        prov2 = df.iloc[[np.argmax(df['datetime'])]]['close']
        ultimovalor.append(float(prov2))
    #Criam um DataFrame "previsoes" com os valores previstos dos próximos 2 minutos das moedas e, a partir disso, a tendência percentual e o sentido da operação
    #O sentido da operação é pode ser classificado como positivo (quando as duas previsões indicam o mesmo sentido, de queda ou subida.)
    #ou negativo (quando as previsões indicam uma queda e uma subida)
    previsoes['Ultimo']=ultimovalor
    previsoes['Dif1']=previsoes[previsoes.columns[0]]-previsoes['Ultimo']
    previsoes['Dif2']=previsoes[previsoes.columns[1]]-previsoes[previsoes.columns[0]]
    previsoes['sentido']=previsoes['Dif1']*previsoes['Dif2']
    previsoes['tendencia']=(previsoes[previsoes.columns[1]]-previsoes['Ultimo'])/previsoes['Ultimo']
    previsoes=previsoes.reset_index()
    #esta parte do loop irá definir qual o montante que será negociado (caso os indicadores de tendencia ou sentido sejam atingidos
    #O montante negociado é proporcional à tendência identificada, podendo variar entre 10 e 1000 dolares
    for i in range(3):
        prov = compute_quantity(coin_value = ultimovalor[i], invest_value = min(1000,max(10,abs(previsoes['tendencia'][i]/0.002)*1000)), significant_digits = 4)
        qtdade.append(prov)
    # Print do datetime atual
    print('-------------------')
    print('Rodada nº',x)
    now = pd.to_datetime('now')
    print(f'{now}')
    #loop interno 3: verifica a regra de tendência e realiza a operação de compra/venda para cada uma das 3 moedas:
    #Caso as condições de compra e venda não forem atingidas, o processo não realiza uma transação
    for i in range(3):
        if previsoes['tendencia'][i] > 0.0005 or (previsoes['sentido'][i]>0 and previsoes['tendencia'][i] > 0):
            # Modelo detectou uma tendência positiva da moeda i
            if previsoes['sentido'][i]>0:
                print(str(ticker[i]),": Está com tendência positiva de ",previsoes['tendencia'][i]*100, "% a tendencia se manteve para as próximas ",qteprev,"previsões.")
            else:
                print(str(ticker[i]),": Está com tendência positiva de ",previsoes['tendencia'][i]*100, "% o sentido das previsões não foram uniformes porém a tendência se manteve superior a 0,05%")

            # Verifica quanto dinheiro tem em caixa
            qtdade_money = how_much_i_have('money', token)
        
            if qtdade_money>0:
                # Se tem dinheiro, tenta comprar o equivalente à quantidade ou o máximo que o dinheiro permitir
                max_qtdade = compute_quantity(coin_value = ultimovalor[i], invest_value = qtdade_money, significant_digits = 4)
                qtdadeuso = min(qtdade[i], max_qtdade)
            
            # Realizando a compra
            print(f'Comprando {str(qtdadeuso)} {ticker[i]}')
            api_post('buy', payload = {'token': token, 'ticker': ticker[i], 'quantity': qtdadeuso})
        
        elif previsoes['tendencia'][i] < -0.0005 or (previsoes['sentido'][i]>0 and previsoes['tendencia'][i] < 0):
            # Modelo detectou uma tendência negativa da moeda i
            if previsoes['sentido'][i]>0:
                print(str(ticker[i]),": Está com tendência negativa de ",previsoes['tendencia'][i]*100, "% a tendencia se manteve para as próximas ",qteprev,"previsões.")
            else:
                print(str(ticker[i]),": Está com tendência negativa de ",previsoes['tendencia'][i]*100, "% o sentido das previsões não foram uniformes porém a tendência se manteve superior a 0,05%")
            # Verifica quanto tem da moeda em caixa
            qtdade_coin = how_much_i_have(ticker[i], token)
        
            if qtdade_coin>0:
                # Se tenho a moeda, vou vender!
                qtdadeuso = min(qtdade_coin, qtdade[i])
                print(f'Vendendo {str(qtdadeuso)} {ticker[i]}')
                api_post('sell', payload = {'token': token, 'ticker': ticker[i], 'quantity':qtdadeuso})
        else:
            # Não faz nenhuma ação, espera próximo loop
            print(str(ticker[i]),": Está sem sentido definido e com tendência neutra de ",previsoes['tendencia'][i]*100,"%. Nenhuma ação realizada")
    x=x+1
    # Print do status após cada iteração
    print(api_post('status', payload = {'token': token}))
    previsoes=[]
    time.sleep(60)

In [ ]:
# Para  não ser impactado por variações da Cripto (positivas ou negativas) enquanto o robô não estiver ativado, o código abaixo vende toda a moeda disponível
for i in range(3):
    qtdade_coin = how_much_i_have(ticker[i], token)
    print(f'Vendendo {str(qtdade_coin)} {ticker[i]}')
    api_post('sell', payload = {'token': token, 'ticker': ticker[i], 'quantity':qtdade_coin})
    print(api_post('status', payload = {'token': token}))